### **6. Интеграция с внешними авторизационными серверами**

#### **Шаг 1: Основы интеграции с внешними провайдерами**

OAuth 2.0 позволяет использовать авторизационные серверы сторонних сервисов, таких как Google, GitHub, Facebook, и др., для аутентификации пользователей вашего приложения. Это удобно, так как пользователи могут войти с помощью существующих учётных записей.

Для интеграции вам понадобится:
- Зарегистрировать приложение у стороннего провайдера.
- Настроить Django для взаимодействия с этим провайдером.

#### **Шаг 2: Пример интеграции с Google OAuth 2.0**

1. **Регистрация приложения на платформе Google**
   - Перейдите в [Google Cloud Console](https://console.cloud.google.com/).
   - Создайте новый проект или выберите существующий.
   - Перейдите в раздел **APIs & Services** → **Credentials**.
   - Нажмите **Create Credentials** → **OAuth 2.0 Client ID**.
     - Укажите тип приложения: **Web application**.
     - Добавьте разрешённые URI:
       - **Authorized redirect URI:** Укажите адрес вашего приложения, например:  
         `http://127.0.0.1:8000/accounts/google/login/callback/` (для локального сервера).
   - Сохраните **Client ID** и **Client Secret** — они понадобятся для настройки.

2. **Установка библиотек для Django**
   Установите библиотеку `django-allauth`, которая поддерживает интеграцию с OAuth 2.0:
   ```bash
   pip install django-allauth
   ```

3. **Настройка проекта**
   В `settings.py` добавьте следующие параметры:

   ```python
   INSTALLED_APPS += [
       'django.contrib.sites',  # Обязательно для allauth
       'allauth',
       'allauth.account',
       'allauth.socialaccount',
       'allauth.socialaccount.providers.google',  # Провайдер Google OAuth
   ]

   SITE_ID = 1  # Стандартный ID для вашего сайта

   # Настройка редиректов
   LOGIN_REDIRECT_URL = '/'  # Куда перенаправлять пользователя после входа
   LOGOUT_REDIRECT_URL = '/'  # Куда перенаправлять после выхода

   # Настройка авторизационного сервера
   SOCIALACCOUNT_PROVIDERS = {
       'google': {
           'APP': {
               'client_id': 'ваш_client_id',
               'secret': 'ваш_client_secret',
               'key': ''
           }
       }
   }
   ```

4. **Добавление маршрутов**
   В файле `urls.py` подключите маршруты `allauth`:
   ```python
   from django.urls import path, include

   urlpatterns = [
       path('accounts/', include('allauth.urls')),  # URL для OAuth 2.0
   ]
   ```

5. **Миграции и запуск**
   Выполните миграции:
   ```bash
   python manage.py migrate
   ```
   Запустите сервер:
   ```bash
   python manage.py runserver
   ```

6. **Тестирование**
   Перейдите по адресу:
   ```
   http://127.0.0.1:8000/accounts/google/login/
   ```
   - Вы будете перенаправлены на страницу входа Google.
   - После авторизации Google вернёт вас на сайт, и пользователь будет автоматически зарегистрирован в базе данных.


#### **Шаг 3: Пример интеграции с GitHub OAuth 2.0**

1. **Регистрация приложения на платформе GitHub**
   - Перейдите в [GitHub Developer Settings](https://github.com/settings/developers).
   - Нажмите **New OAuth App**.
   - Укажите настройки:
     - **Application name:** Название приложения.
     - **Homepage URL:** Например, `http://127.0.0.1:8000/`.
     - **Authorization callback URL:** Укажите:  
       `http://127.0.0.1:8000/accounts/github/login/callback/`.
   - Сохраните приложение. Вы получите **Client ID** и **Client Secret**.

2. **Настройка провайдера GitHub в Django**
   - Убедитесь, что библиотека `django-allauth` уже установлена.
   - Добавьте в `INSTALLED_APPS` провайдер GitHub:
     ```python
     'allauth.socialaccount.providers.github',
     ```
   - В `settings.py` добавьте параметры для GitHub:
     ```python
     SOCIALACCOUNT_PROVIDERS = {
         'github': {
             'APP': {
                 'client_id': 'ваш_client_id',
                 'secret': 'ваш_client_secret',
                 'key': ''
             }
         }
     }
     ```

3. **Тестирование**
   Перейдите по адресу:
   ```
   http://127.0.0.1:8000/accounts/github/login/
   ```
   После авторизации GitHub создаст пользователя в базе данных Django.


#### **Шаг 4: Настройка доступа к данным пользователя**

OAuth позволяет запрашивать дополнительные данные пользователя, такие как email, имя, аватар.

1. Для Google:
   В настройках Google Console убедитесь, что API **People API** активирован. Теперь вы можете запросить такие данные:
   - Имя.
   - Email.
   - Аватар.

2. Для GitHub:
   GitHub предоставляет доступ к данным профиля, например:
   - Имя.
   - Логин.
   - Репозитории.
   - Аватар.

3. **Обработка данных в Django:**
   После успешного входа данные сохраняются в `SocialAccount`. Например, для получения email:
   ```python
   from allauth.socialaccount.models import SocialAccount

   def get_user_email(user):
       social_account = SocialAccount.objects.filter(user=user, provider='google').first()
       if social_account:
           return social_account.extra_data.get('email')
       return None
   ```


#### **Вывод**


Интеграция с внешними авторизационными серверами упрощает процесс аутентификации, повышает безопасность и улучшает пользовательский опыт. Выбор между Google, GitHub или другими провайдерами зависит от вашей аудитории и функциональных требований.